### Ingestion del archivo "language_role.json"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de spark

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
languages_roles_schema = StructType(fields=[
    StructField("roleId", IntegerType(), False),
    StructField("languageRole", StringType(), False)
])

In [0]:
languages_roles_df = spark.read \
                .schema(languages_roles_schema) \
                .option("multiLine", True) \
                .json(f"{bronze_folder_path}/{v_file_date}/language_role.json")

In [0]:
display(languages_roles_df)

#### Paso 2 - Renombrar las columnas y añadir nuevas
1. "roleId" renombrar a "role_id"
2. "languageRole" renombrar a "language_role"
3. agregar las columnas "ingestion_date" y "environment"

In [0]:
languages_roles_finals_df = add_ingestion_date(languages_roles_df) \
                                .withColumnsRenamed({"roleId": "role_id",
                                              "languageRole": "language_role"}) \
                                .withColumn("environment", lit(v_environment)) \
                                .withColumn("file_date", lit(v_file_date))

In [0]:
display(languages_roles_finals_df)

#### Paso 3 - Eliminar columnas no deseadas

In [0]:
#movies_cast_final_df = movies_casts_with_columns_df.drop(col("genderId"), col("castOrder"))

In [0]:
#display(movies_cast_final_df)

#### Paso 4 - Escribir en parquet

In [0]:
languages_roles_finals_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages_roles")

In [0]:
display(spark.read.format("delta").load("/mnt/moviehistory2025/silver/languages_roles"))

In [0]:
dbutils.notebook.exit("success")